# explore sketch object similarities

The purpose of this notebook is to:
* Measure how much more similar successive sketches of the SAME object are than sketches of DIFFERENT objects drawn in consecutive repetitions
* Measure how much more similar sketches of the SAME object are within a repetition (a proxy for resemblance) than arbitrarily sampled sketches from within a repetition. Examine how this similarity changes over time. 

In [1]:
import os, sys
## add helpers to python path
if os.path.join('..','helpers') not in sys.path:
    sys.path.append(os.path.join('..','helpers'))

import pymongo as pm
import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import requests
import re
from io import BytesIO
from PIL import Image, ImageFilter
import object_mask_utils as u
import socket
import glob
from scipy.stats import entropy

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

from skimage import io, img_as_float
import base64

from IPython.core.pylabtools import figsize, getfigs

from IPython.display import clear_output
import importlib
import time

from collections import Counter
import operator

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

### setup paths

In [3]:
## which experiment do you want to analyze? options: refgame1.2, refgame2.o
curr_exp = 'refgame1.2'

## directory & file hierarchy
proj_dir = os.path.abspath('../../')
analysis_dir = os.getcwd()
data_dir = os.path.join(proj_dir,'data')
experiment_dir = os.path.join(data_dir, 'experiment', curr_exp)
feat_dir = os.path.join(data_dir, 'features', curr_exp)

## paths to visual features & metadata
path_to_feats = os.path.join(feat_dir, 'FEATURES_vgg_FC6.npy')
path_to_meta = os.path.join(feat_dir, 'METADATA.csv')

## import dictionaries that map between shapenet ids and graphical conventions naming scheme
importlib.reload(u)
G2S = u.GC2SHAPENET
S2G = u.SHAPENET2GC